# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kodinsk,58.6881,99.1844,-26.42,100,95,0.25,RU,1708890852
1,1,pahrump,36.2083,-115.9839,19.03,35,100,0.89,US,1708890792
2,2,hermanus,-34.4187,19.2345,17.15,64,5,3.31,ZA,1708890852
3,3,rio tinto,41.1826,-8.5581,13.77,90,75,5.81,PT,1708890852
4,4,grytviken,-54.2811,-36.5092,4.32,94,100,5.52,GS,1708890852


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
coordinates_df = city_data_df[["Lat","Lng","City","Humidity"]]

# Display the map
map_plot_1 = coordinates_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.7,
    alpha = 0.5,
    color = "City"
)
map_plot_1

ImportError: Geographic projection support requires: cartopy, geoviews, pyproj.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_ideal = city_data_df.dropna()
city_data_df_ideal = city_data_df_ideal[(city_data_df_ideal['Max Temp']>=21) & (city_data_df_ideal['Max Temp']<=27)]


# Display sample data
city_data_df_ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,adamstown,-25.0660,-130.1015,26.46,76,96,5.27,PN,1708890853
11,11,rongelap,11.1635,166.8941,26.58,81,85,6.69,MH,1708890854
12,12,tolanaro,-25.0319,46.9987,25.04,88,75,4.12,MG,1708890854
23,23,howard springs,-12.4970,131.0470,26.95,96,98,1.25,AU,1708890857
26,26,port mathurin,-19.6833,63.4167,26.40,77,85,9.84,MU,1708890858
...,...,...,...,...,...,...,...,...,...,...
545,545,sao gabriel da cachoeira,-0.1303,-67.0892,23.89,94,75,2.06,BR,1708891040
547,547,namibe,-15.1961,12.1522,24.64,86,100,0.78,AO,1708891040
549,549,tarariras,-34.2833,-57.6167,23.51,53,100,7.30,UY,1708891041
550,550,viqueque,-8.8673,126.3658,24.98,90,98,2.08,TL,1708891041


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_ideal[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:,['Hotel Name']] = ''
hotel_df
# Display sample data
hotel_df

C:\Users\miguel.vargas\AppData\Local\Temp\ipykernel_2592\3394489250.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.loc[:,['Hotel Name']] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
5,adamstown,PN,-25.0660,-130.1015,76,
11,rongelap,MH,11.1635,166.8941,81,
12,tolanaro,MG,-25.0319,46.9987,88,
23,howard springs,AU,-12.4970,131.0470,96,
26,port mathurin,MU,-19.6833,63.4167,77,
...,...,...,...,...,...,...
545,sao gabriel da cachoeira,BR,-0.1303,-67.0892,94,
547,namibe,AO,-15.1961,12.1522,86,
549,tarariras,UY,-34.2833,-57.6167,53,
550,viqueque,TL,-8.8673,126.3658,90,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
hotel = ''
params = {
    "filter":'filters',
    "bias":'bias',
    'categories': 'accommodation.hotel',
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lng = row['Lng']
    Lat = row['Lat']    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params['categories'] = f'accommodation.hotel'
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
        # Convert the API response to JSON format
    name_address = requests.get(base_url,params=params).json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
rongelap - nearest hotel: No hotel found
tolanaro - nearest hotel: Hôtel Mahavokey
howard springs - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
darwin - nearest hotel: Mantra Pantanas Darwin
piriapolis - nearest hotel: Esmeralda
wewak - nearest hotel: Village Inn
miri - nearest hotel: Kenyalang Suites Miri
antofagasta - nearest hotel: Hotel Colón
rio grande - nearest hotel: Hotel Vila Moura Executivo
hadibu - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
tindouf - nearest hotel: محمد بوسبي
dwarka - nearest hotel: The Dwarika Hotel
wedgefield - nearest hotel: No hotel found
saint-philippe - nearest hotel: Le Baril
saint-pierre - nearest hotel: Tropic Hotel
tucuma - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
mount isa - nearest hotel: Ibis Styles
ko lanta - nearest hotel: Kroua Lanta
tadine - nearest hotel: Hôtel

,City,Country,Lat,Lng,Humidity,Hotel Name
5,adamstown,PN,-25.0660,-130.1015,76,No hotel found
11,rongelap,MH,11.1635,166.8941,81,No hotel found
12,tolanaro,MG,-25.0319,46.9987,88,Hôtel Mahavokey
23,howard springs,AU,-12.4970,131.0470,96,No hotel found
26,port mathurin,MU,-19.6833,63.4167,77,Escale Vacances
...,...,...,...,...,...,...
545,sao gabriel da cachoeira,BR,-0.1303,-67.0892,94,Hotel Deus Me Deu
547,namibe,AO,-15.1961,12.1522,86,Hotel Chik Chik Namibe
549,tarariras,UY,-34.2833,-57.6167,53,No hotel found
550,viqueque,TL,-8.8673,126.3658,90,Hotel Wailakurini


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
coordinates_df = hotel_df[["Lat","Lng","City","Humidity","Hotel Name", "Country"]]

# Display the map
map_plot_1 = coordinates_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.7,
    alpha = 0.5,
    color = "City",
    hover_cols = "all"
)
map_plot_1

ImportError: Geographic projection support requires: cartopy, geoviews, pyproj.